# Test Data Preprocessing

This notebook contains data cleaning and feature engineering for the test data. It's essentially a carbon copy of training_data_preprocessing.ipynb using the same methods of preprocessing my test data.

In [2]:
# Import libraries
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
from skrub import SimilarityEncoder
from fancyimpute import IterativeImputer

In [3]:
# Importing the custom functions
import sys
import os 
sys.path.append(os.path.abspath('../src'))
from utils import *

In [22]:
# Set max display options so that I can see everything I need to see
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 200)

In [32]:
df = pd.read_csv('../data/processed/test_set.csv')

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     14850 non-null  int64  
 1   amount_tsh             4440 non-null   float64
 2   date_recorded          14850 non-null  object 
 3   funder                 13980 non-null  object 
 4   gps_height             9639 non-null   float64
 5   installer              13973 non-null  object 
 6   longitude              14393 non-null  float64
 7   latitude               14393 non-null  float64
 8   wpt_name               14850 non-null  object 
 9   num_private            14850 non-null  int64  
 10  basin                  14850 non-null  object 
 11  subvillage             14751 non-null  object 
 12  region                 14850 non-null  object 
 13  region_code            14850 non-null  int64  
 14  district_code          14850 non-null  int64  
 15  lg

---

## Dealing with Missing Values

No exploration or overview of missing data here, just prepping the data for testing my classification models.

### Strategy 1: Naive approach - Dropping all records with missing values

In [35]:
# Dropping the columns that are not needed based on my EDA
df_test = df.drop(columns=['id', 'wpt_name', 'num_private', 'subvillage', 'ward', 'recorded_by', 'scheme_name', 
                            'scheme_management', 'water_quality', 'waterpoint_type_group', 'quantity_group', 'region_code', 
                            'extraction_type', 'extraction_type_group', 'payment', 'source_class', 'source_type'])

The 0's that are not imputed remain as 0.

In [36]:
# Make a copy of df_train to test dropping all nulls
strat1 = df_test.copy()

In [37]:
# amount_tsh has such as a large number of missing values that it is not 
# worth keeping for this approach as too many rows would be dropped along with
# its missing values
strat1 = strat1.drop(columns=['amount_tsh'])

In [38]:
# Drop all null values
strat1 = strat1.dropna()

In [39]:
strat1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8069 entries, 0 to 14849
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date_recorded          8069 non-null   object 
 1   funder                 8069 non-null   object 
 2   gps_height             8069 non-null   float64
 3   installer              8069 non-null   object 
 4   longitude              8069 non-null   float64
 5   latitude               8069 non-null   float64
 6   basin                  8069 non-null   object 
 7   region                 8069 non-null   object 
 8   district_code          8069 non-null   int64  
 9   lga                    8069 non-null   object 
 10  population             8069 non-null   float64
 11  public_meeting         8069 non-null   object 
 12  permit                 8069 non-null   object 
 13  construction_year      8069 non-null   float64
 14  extraction_type_class  8069 non-null   object 
 15  manageme

In [40]:
# Export to csv
strat1.to_csv('../data/processed/test_all_nulls_dropped.csv', index=False)

### Strategy 2: Imputation

Only the nearby samples imputation will be done here. See training_data_preprocessing.ipynb for more explanations on everything.

In [41]:
def find_k_nearest_by_coordinates(
        samples_df: pd.DataFrame,
        k_nearest: int,
        longitude: float,
        latitude: float
) -> pd.DataFrame:
    geo_coordinates = samples_df[['longitude', 'latitude']]
    target_coord = np.float32([longitude, latitude])
    distances = np.sqrt(np.sum(np.power(target_coord - geo_coordinates, 2), axis=1))
    min_distance_indices = np.argpartition(distances, k_nearest)[1: k_nearest+1]
    return samples_df.iloc[min_distance_indices]


def impute_zeros_by_nearby_samples(
        samples_df: pd.DataFrame,
        location_column: str,
        target_column: str,
        std_threshold: float or None
) -> int:
    num_imputed = 0

    for area in samples_df[location_column].unique():
        row_ids = samples_df[location_column] == area
        target_values = samples_df.loc[row_ids, target_column]

        if target_values.shape[0] > 1:
            non_zero_ids = target_values > 0

            if non_zero_ids.sum() > 0:
                non_zero_values = target_values[non_zero_ids]

                if std_threshold is not None and np.std(non_zero_values) > std_threshold:
                    continue

                zero_ids = np.invert(non_zero_ids)
                target_values[zero_ids] = non_zero_values.mean()
                samples_df.loc[row_ids, target_column] = target_values
                num_imputed += zero_ids.sum()
    return num_imputed

#### amount_tsh, population, gps_height

For these imputation methods to work I have to convert the NaN values in my numerical categories back to 0.

In [42]:
# Replace 'np.nan' values in 'construction_year' with 0
df['construction_year'] = df['construction_year'].replace(np.nan, 0)

In [43]:
# Replace np.nan values in 'amount_tsh' with 0
df['amount_tsh'] = df['amount_tsh'].replace(np.nan, 0)

In [44]:
# Replace np.nan values in 'gps_height' with 0
df['gps_height'] = df['gps_height'].replace(np.nan, 0)

In [45]:
# Replace np.nan values in 'population' with 0
df['population'] = df['population'].replace(np.nan, 0)

The 0's that are not imputed remain as 0.

In [46]:
amount_tsh_std_threshold = 50

num_amount_tsh_missing = (df["amount_tsh"] == 0).sum()
num_imputed = impute_zeros_by_nearby_samples(
    samples_df=df,
    location_column='subvillage',
    target_column='amount_tsh',
    std_threshold=amount_tsh_std_threshold
)
print(f'Imputed {num_imputed}/{num_amount_tsh_missing} missing "amount_tsh" values')

Imputed 751/10410 missing "amount_tsh" values


In [47]:
population_std_threshold = 50

num_amount_tsh_missing = (df["population"] == 0).sum()
num_imputed = impute_zeros_by_nearby_samples(
    samples_df=df,
    location_column='subvillage',
    target_column='population',
    std_threshold=population_std_threshold
)
print(f'Imputed {num_imputed}/{num_amount_tsh_missing} missing "population" values')

Imputed 273/5453 missing "population" values


In [48]:
location_columns = ['subvillage', 'ward', 'lga', 'district_code']

for location_column in location_columns:
    num_gps_height_missing = (df["gps_height"] == 0).sum()
    num_imputed = impute_zeros_by_nearby_samples(
        samples_df=df,
        location_column=location_column,
        target_column='gps_height',
        std_threshold=None
    )
    print(f'Imputed {num_imputed}/{num_gps_height_missing} missing "gps_height"')

Imputed 1070/5211 missing "gps_height"
Imputed 2639/4277 missing "gps_height"
Imputed 1862/1856 missing "gps_height"
Imputed 25/25 missing "gps_height"


In [49]:
k_neighbors = 25
gps_height_zero_ids = df['gps_height'] == 0
gps_zero_samples = df[gps_height_zero_ids]
gps_heights = []

for _, sample in gps_zero_samples.iterrows():
    longitude = df['longitude']
    latitude = df['latitude']
    nearest_samples = find_k_nearest_by_coordinates(
        samples_df=df,
        k_nearest=k_neighbors,
        longitude=longitude,
        latitude=latitude
    )
    non_zero_gps_height_values = nearest_samples.loc[nearest_samples['gps_height'] != 0, 'gps_height']
    gps_heights.append(non_zero_gps_height_values.mean())

df.loc[df['gps_height'] == 0, 'gps_height'] = gps_heights
print(f'gps_height == 0: {(df["gps_height"] == 0).sum()} After K-NN method')

gps_height == 0: 0 After K-NN method


---

## Feature Engineering on the Remaining Features

Before modeling, the rest of the features could also use a bit of modification.

#### **permit and public_meeting**

Encoding these two boolean categories.

In [50]:
# Encoding the booleans in these two categories to 0 (False)m 1(True), and -1 (NaN) 
df['public_meeting'] = df['public_meeting'].fillna('Unknown')
df['public_meeting'] = df['public_meeting'].replace({'False': 0, 'True': 1, 'Unknown': -1})

df['permit'] = df['permit'].fillna('Unknown')
df['permit'] = df['permit'].replace({'False': 0, 'True': 1, 'Unknown': -1})

#### **Creating a pump_age feature**

The features 'construction_year' and 'date_recorded' can be used to create a feature that tells us the duration a pump has been in operation. However, 'construction_year' has a lot of missing values. In such cases I'll set pump_age to -1.

In [51]:
# Creating a new feature that represents the age of the pump
df['construction_year'] = df['construction_year'].replace(0, 10000) # Replacing 0 with 10000 to capture invalid pump ages
df['pump_age'] = pd.DatetimeIndex(df['date_recorded']).year - df['construction_year']

invalid_pump_age = df['pump_age'] < 0
df.loc[invalid_pump_age, 'pump_age'] = -1 

#### **Creating a time recorded feature with lower dimensionality 'season'**

Insteaf of date recorded or even month recorded I'll condense it down to seasons I discovered while doing EDA - essentially a split between rainy and dry seasons.

- ShortDry: January-February
- LongRainy: March-May
- LongDry: June-October
- ShortRainy: November-December

In [52]:
# Dictionary matching months to their corresponding seasons
# 0: ShortDry, 1: LongRainy, 2: LongDry, 3: ShortRainy
seasons = {
    1: 0, 2: 0,
    3: 1, 4: 1, 5: 1,
    6: 2, 7: 2, 8: 2, 9: 2, 10: 2,
    11: 3, 12: 3,
}

# Creating the 'season' column
df['season'] = pd.DataFrame({'Month': pd.DatetimeIndex(df['date_recorded']).month})
df['season'] = df['season'].apply(lambda month: seasons[month])

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 42 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     14850 non-null  int64  
 1   amount_tsh             14850 non-null  float64
 2   date_recorded          14850 non-null  object 
 3   funder                 13980 non-null  object 
 4   gps_height             14850 non-null  float64
 5   installer              13973 non-null  object 
 6   longitude              14393 non-null  float64
 7   latitude               14393 non-null  float64
 8   wpt_name               14850 non-null  object 
 9   num_private            14850 non-null  int64  
 10  basin                  14850 non-null  object 
 11  subvillage             14751 non-null  object 
 12  region                 14850 non-null  object 
 13  region_code            14850 non-null  int64  
 14  district_code          14850 non-null  int64  
 15  lg

In [55]:
# Export to csv file
df.to_csv('../data/final/test_cleaned.csv', index=False)